In [18]:
# | Field Name | Data Type | Description (Vietnamese) | Description (English) | Business Rules | Example |
# |------------|-----------|--------------------------|----------------------|----------------|---------|
# | transaction_key | int | Khóa chính surrogate | Primary surrogate key | PK, NOT NULL, UNIQUE | 1001 |
# | customer_key | int | Khóa khách hàng | Customer foreign key | FK to DIM_CUSTOMER | 12345 |
# | product_key | int | Khóa sản phẩm | Product foreign key | FK to DIM_PRODUCT | 567 |
# | location_key | int | Khóa địa điểm | Location foreign key | FK to DIM_LOCATION | 789 |
# | event_key | int | Khóa sự kiện | Event foreign key | FK to DIM_EVENT | 101 |
# | involved_party_key | int | Khóa bên liên quan | Involved party foreign key | FK to DIM_INVOLVED_PARTY | 234 |
# | condition_key | int | Khóa điều kiện | Condition foreign key | FK to DIM_CONDITION | 345 |
# | application_key | int | Khóa ứng dụng | Application foreign key | FK to DIM_APPLICATION | 456 |
# | asset_key | int | Khóa tài sản | Asset foreign key | FK to DIM_ASSET | 678 |
#------------------------------------------------------------------
#    KEY 
#------------------------------------------------------------------
# | transaction_id | string | Mã giao dịch duy nhất | Unique transaction ID | NOT NULL, UNIQUE | TXN20240315143025001 |
# | reference_number | string | Số tham chiếu | Reference number | Max 50 chars | REF123456789 |
# | transaction_type | string | Loại giao dịch | Transaction type | NOT NULL | Chuyển khoản, Rút tiền, Nạp tiền,  Nhận Tiền |
# | transaction_category | string | Danh mục giao dịch | Transaction category | NOT NULL | Nội bộ, Liên ngân hàng, Quốc tế |
# | transaction_subcategory | string | Danh mục phụ | Transaction subcategory | Max 100 chars | Chuyển khoản cùng ngân hàng |
# | transaction_amount | decimal | Số tiền giao dịch | Transaction amount | NOT NULL, >= 0 | 1000000 |
# | fee_amount | decimal | Phí giao dịch | Fee amount | >= 0 | 11000 |
# | tax_amount | decimal | Thuế | Tax amount | >= 0 | 1100 |
# | net_amount | decimal | Số tiền thực nhận | Net amount | NOT NULL | 987900 |
# | currency | string | Loại tiền tệ | Currency code | ISO 4217, NOT NULL | VND |
# | transaction_status | string | Trạng thái giao dịch | Transaction status | NOT NULL | Thành công, Thất bại, Đang xử lý |
# | channel | string | Kênh giao dịch | Transaction channel | NOT NULL | ATM, Mobile App, Internet Banking |
# | description | string | Mô tả giao dịch | Transaction description | Max 500 chars | Chuyển khoản cho thuê nhà |
# | created_timestamp | timestamp | Thời gian tạo | Creation timestamp | NOT NULL | 2024-03-15 14:30:25.123 |
# | processed_timestamp | timestamp | Thời gian xử lý | Processing timestamp | >= created_timestamp | 2024-03-15 14:30:27.456 |
# | updated_timestamp | timestamp | Thời gian cập nhật cuối | Last update timestamp | >= created_timestamp | 2024-03-15 14:31:00.789 |



In [ ]:
from datetime import datetime, timedelta
from faker import Faker
import pandas as pd
import random
import uuid
import psycopg2
from sqlalchemy import create_engine
import os 
from dotenv import load_dotenv

class getInfoTransaction:
    def __init__(self, conn_params):
        self.engine = create_engine(
            f"postgresql+psycopg2://{conn_params['user']}:{conn_params['password']}@{conn_params['host']}:{conn_params['port']}/{conn_params['dbname']}"
        )

    def _query(self, sql: str) -> pd.DataFrame:
        try:
            conn = self.engine.raw_connection()   
            try:
                df = pd.read_sql_query(sql, conn)
            finally:
                conn.close()  
            return df
        except Exception as e:  
            print(f"Error executing query: {e}")
            return pd.DataFrame()

    def getCustomer(self):
        df = self._query("SELECT customer_key FROM banking_dw.dim_customer")
        return df

    def getLocation(self):
        df = self._query("SELECT location_key   FROM banking_dw.dim_location")
        return df

    def getApplication(self):
        df = self._query("SELECT application_key FROM banking_dw.dim_application")
        return df

    def getAccount(self):
        df = self._query("SELECT account_key , account_number , customer_key FROM banking_dw.dim_account")
        return df

class GenerationTranaction :
    def __init__(self , conn_params):
        self.conn_params = conn_params
        self.account = getInfoTransaction(conn_params).getAccount()
        self.location = getInfoTransaction(conn_params).getLocation()
        self.application = getInfoTransaction(conn_params).getApplication()
        self.transaction_types = ["Chuyển khoản", "Rút tiền", "Nạp tiền", "Nhận Tiền"]
        self.transaction_category = ["Nội bộ", "Liên ngân hàng", "Quốc tế"]
        self.transaction_status = ["Thành công", "Thất bại"]
        self.channel = ["ATM", "Mobile App", "Internet Banking"]
        
    def get_customer_key(self , account_key):
        df = self.account[self.account["account_key"] == account_key]
        if not df.empty:
            return df.iloc[0]["customer_key"]
        return None
    
    # tinh tien net nhan duoc khi thuc hien giao dich
    def amount(self ,transaction_types , transaction_status,  transaction_amount , fee_amount , tax_amount , ):
        if transaction_status == "Thất bại":
            return 0.0
        else:
            if transaction_types == "Rút tiền":
                return round(transaction_amount + fee_amount + tax_amount , 2)
            elif transaction_types == "Nạp tiền":
                return round(transaction_amount)
            elif transaction_types == "Chuyển khoản":  
                return  round(transaction_amount + fee_amount + tax_amount , 2)
            elif transaction_types == "Nhận Tiền":
                return round(transaction_amount)
        
        
    def transaction_account_number(self , transaction_category ):
        if transaction_category == "Nội bộ":
            return random.choice(self.account["account_number"])
        elif transaction_category == "Liên ngân hàng":
            return f"LB{random.randint(100000, 999999)}"
        elif transaction_category == "Quốc tế":
            return f"QT{random.randint(100000, 999999)}"
        
    
    
    
    def generator_data_transaction(self):
        
        account_key   =  int(random.choice(self.account["account_key"]))
        customer_key  =  int(self.get_customer_key(account_key))
        location_key  =  int(random.choice(self.location["location_key"]))
        event_key = uuid.uuid4().int >> 64
        application_key = int(random.choice(self.account["account_key"]))
        transaction_id = f"TXN{datetime.now().strftime('%Y%m%d%H%M%S')}{random.randint(100, 999)}"
        reference_number = f"REF{uuid.uuid4().hex[:10].upper()}"
        transaction_type = random.choice(self.transaction_types)
        transaction_category = random.choice(self.transaction_category)
        transaction_amount = round(random.uniform(10000, 10000000), 2)
        transaction_status = random.choice(self.transaction_status)
        fee_amount = round(transaction_amount * random.uniform(0.001, 0.01), 2)
        tax_amount = round(fee_amount * 0.1, 2)
        net_amount = self.amount(transaction_type,transaction_status,transaction_amount, fee_amount, tax_amount) 
        currency = "VND"
        account_number = self.transaction_account_number(transaction_category)

        channel = random.choice(self.channel)
        description = f"Giao dịch {transaction_type} qua {channel}"
        created_timestamp = datetime.now()
        processed_timestamp = created_timestamp + timedelta(seconds=random.randint(1, 300))
        updated_timestamp = processed_timestamp + timedelta(seconds=random.randint(1, 300))

        
        return {
            "account_key": account_key,
            "customer_key": customer_key,
            "location_key": location_key,
            "event_key": event_key,
            "application_key": application_key,
            "transaction_id": transaction_id,
            "reference_number": reference_number,
            "transaction_type": transaction_type,
            "transaction_category": transaction_category,
            "transaction_amount": transaction_amount,
            "fee_amount": fee_amount,
            "tax_amount": tax_amount,
            "net_amount": net_amount,
            "currency": currency,
            "account_number": account_number,
            "transaction_status": transaction_status,
            "channel": channel,
            "description": description,
            "created_timestamp": created_timestamp,
            "processed_timestamp": processed_timestamp,
            "updated_timestamp": updated_timestamp
        }
    
    def print(self):
        print(self.generator_data_transaction())
        
        
    
    
if __name__ == "__main__":  
    load_dotenv()
    
    conn_params = {
        "host": os.getenv("DB_HOST"),
        "port": os.getenv("DB_PORT"),
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASS")
    }
    
    info_transaction = GenerationTranaction(conn_params)
    info_transaction.print()

{'account_key': 19, 'customer_key': 9, 'location_key': 7, 'event_key': 2266322082415461315, 'application_key': 24, 'transaction_id': 'TXN20250904163220549', 'reference_number': 'REF44ED7C15FF', 'transaction_type': 'Chuyển khoản', 'transaction_category': 'Liên ngân hàng', 'transaction_amount': 5204812.88, 'fee_amount': 48518.19, 'tax_amount': 4851.82, 'net_amount': 5258182.89, 'currency': 'VND'}


C:\Users\admin\AppData\Local\Temp\ipykernel_17396\855600106.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)
